In [1]:
import pandas as pd
import csv

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

abbreviations = {abbreviation: state for state, abbreviation in states.items()}

## `preprocess_state_sales.py`

In [2]:
df2019 = pd.read_csv('../data/raw/ev_registrations_2019.csv')

df2019 = df2019[:-1]

df2019.index = df2019['State'].apply(lambda x: abbreviations[x])

df2019['2019 EVs'] = df2019['EV Registrations 2019'].apply(int)
df2019['2019 EVPercent'] = df2019['PercentTotalEV 2019'].apply(lambda x: float(x[:-1]) / 100)

df2019 = df2019.drop(columns=['State', 'EV Registrations 2019', 'PercentTotalEV 2019'])

In [3]:
df2021 = pd.read_csv('../data/raw/ev_registrations_2021.csv')

df2021 = df2021[:-1]

df2021.index = df2021['State'].apply(lambda x: abbreviations[x])

df2021['2021 EVs'] = df2021['EVRegistrations 2021'].apply(lambda x: int(x.replace(',', '')))
df2021['2021 EVPercent'] = df2021['PercentEVDistribution2021'].apply(lambda x: float(x[:-1]) / 100)

df2021 = df2021.drop(columns=['State', 'EVRegistrations 2021', 'PercentEVDistribution2021'])

In [4]:
sales = pd.merge(df2019, df2021, on='State')
sales.to_csv('../data/interim/state_sales.csv')

## `preprocess_model_sales.py`

In [5]:
def process_model_rows(s):
    # determine left and right "boundaries" of non-sales
    left = -1
    right = len(s)
    for i in range(len(s)):  # normally we would use enumerate, but enumerating in reverse doesn't exist
        x = s[i]
        if x != '-':
            left = i
            break

    for i in range(len(s) - 1, -1, -1):
        x = s[i]
        if x != '-':
            right = i
            break

    # if a '-' entry is between the left and right cutoffs, we assume missing data
    s = [
        float(x.replace(',', '')) if x != '-'
        else (0 if i < left or i > right else None)
        for i, x in enumerate(s)
    ]

    return s

def clean_hev(file_name):
    vehicle_data = []

    with open(file_name, 'r') as fin:
        sale_data = csv.reader(fin)
        *header, _ = next(sale_data) # skip header row, but keep information, except total

        for row in sale_data:
            vehicle, *data, _ = row # we need information about the vehicle and the yearly data, and can skip the total
            vehicle_data.append([vehicle, *process_model_rows(data)])

    df = pd.DataFrame(vehicle_data, columns=header)
    df.index = df['Vehicle']
    df['Type'] = 'HEV'
    df = df.drop('Total')
    df = df.drop(columns=['Vehicle'])

    return df

def clean_pev(file_name):
    vehicle_data = []

    with open(file_name, 'r') as fin:
        sale_data = csv.reader(fin)
        *header, _ = next(sale_data) # skip header row, but keep information, except total

        for row in sale_data:
            vehicle, ev_type, *data, _ = row # we need information about the vehicle and the yearly data, and can skip the total
            vehicle_data.append([vehicle, ev_type, *process_model_rows(data)])

    df = pd.DataFrame(vehicle_data, columns=header)
    df.index = df['Vehicle']
    df = df.drop('Total')
    df = df.drop(columns=['Vehicle'])

    return df


hev_df = clean_hev('../data/raw/hev_sales.csv')
pev_df = clean_pev('../data/raw/pev_sales.csv')

model_df = pd.concat([hev_df, pev_df])
model_df = model_df[['Type'] + [str(x) for x in range(1999, 2019 + 1)]]

model_df.to_csv('../data/interim/model_sales.csv')

## `income_preprocess.py`

In [6]:
df = pd.read_csv('../data/raw/median_income.csv')
# Keep only median income columns
temp_header = df.columns + ' ' + df.iloc[0]
df.columns = temp_header

seen = set()
new_columns = []
for column in df.columns:
    if 'Standard error' in column: continue
    year = column.split(" ")[0]
    if year in seen: continue
    else:
        new_columns.append(column)
        seen.add(year)

df = df[new_columns]

# Reformat
df = df.drop(df.index[0])
df = df.drop(df.index[0])
df = df.rename({'State State': 'State'}, axis=1)
df.index = df['State']
df = df.drop(columns=['State'])
df.columns = df.columns[::-1]

df.columns = [col.split(" ")[0] for col in df.columns]
df = df.applymap(lambda x: int(x.replace(",", "")))

df.to_csv("../data/interim/income.csv")

## `populations.py`

In [7]:
df1 = pd.read_csv('../data/raw/population_p1.csv')
df1 = df1.drop(df1.index[0])
df1 = df1.rename({'Area Name': 'State'}, axis=1)
df1.index = df1['State']
df1 = df1.drop(columns=['State'])
df1 = df1.loc[abbreviations.keys()]

In [8]:
df2 = pd.read_csv('../data/raw/population_p2.csv')
df2 = df2.drop(df2.index[0])
df2 = df2.drop(columns=['Estimates Base (4/1/2010)'])
df2 = df2.rename({'Area': 'State'}, axis=1)
df2.index = df2['State']
df2 = df2.drop(columns=['State'])
df2 = df2.loc[abbreviations.keys()]

In [9]:
dfpop = pd.merge(df1, df2, on='State')
dfpop = dfpop.applymap(lambda x: int(x.replace(",", "")))
dfpop.to_csv('../data/interim/populations.csv')

## `state_year_preprocess.py`

In [10]:
def generate_records(file_name, data=None):
    if not data:
        data = []

    with open(file_name) as fin:
        reader = csv.DictReader(fin)
        for data_dict in reader:
            state = data_dict.pop('State')
            if state == 'US': continue
            if state not in states: state = abbreviations[state]
            for year, value in data_dict.items():
                data.append((state, int(year), float(value)))

    return data

field_names = ['Gasoline Price', 'Median Income', 'Population', 'Renewable Energy Use', 'Total Energy Use', 'Transportation Energy Use']
file_names = ['../data/raw/gasoline_price.csv', '../data/interim/income.csv', '../data/interim/populations.csv', '../data/raw/renewable_energy_use.csv', '../data/raw/total_energy_use.csv', '../data/raw/transportation_energy_use.csv']

dfs = []
for field_name, file_name in zip(field_names, file_names):
    records = generate_records(file_name)
    df = pd.DataFrame.from_records(records)
    df.columns = ['State', 'Year', field_name]
    dfs.append(df)

from functools import reduce
socioeconomic_df = reduce(lambda x, y: pd.merge(x, y, on=['State', 'Year']), dfs)
socioeconomic_df = socioeconomic_df.set_index(['State', 'Year'])
socioeconomic_df.to_csv('../data/processed/socioeconomic.csv')